# Eigenes Environment aufbauen

In [2]:
!pip install gym pandas numpy torch bs4 request yfinance


import gym
import pandas as pd
import numpy as np
import os.path
from functools import reduce

from gym import spaces
import torch


import bs4 as bs
import requests
import yfinance as yf
import datetime
import time
import os
import pandas_datareader as web
import warnings
warnings.filterwarnings('ignore')

class MyDataset():

    def __init__(self):
        self.url = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
        self.stocks_fname = "sp500_closefull.csv"
        self.start = datetime.datetime(2010, 1, 1)
        self.stop = datetime.datetime.now()
        self.Ntest = 1000
        self.now = time.time()

    def get_train_test(self) -> pd.DataFrame:

        start = self.start
        end = self.stop


        if not os.path.isfile(self.stocks_fname):
          resp = requests.get(self.url)
          soup = bs.BeautifulSoup(resp.text, 'lxml')
          table = soup.find('table', {'class': 'wikitable sortable'})
          tickers = []

          for row in table.findAll('tr')[1:]:
              ticker = row.findAll('td')[0].text
              tickers.append(ticker)

          tickers = [s.replace('\n', '') for s in tickers]
          data = yf.download(tickers, start=start, end=end)
          data['Adj Close'].to_csv(self.stocks_fname)

        df0 = pd.read_csv(self.stocks_fname, index_col=0, parse_dates=True)

        df_spy = yf.download("SPY", start=start, end=end)

        #df_spy = web.DataReader("SPY", "yahoo", start=start, end=end)
        df_spy = df_spy.loc[:, ['Adj Close']]
        df_spy.columns = ['SPY']
        df0 = pd.concat([df0, df_spy], axis=1)


        #df0.dropna(axis=1, how='all', inplace=True)
        df0.dropna(axis=0, how='all', inplace=True)
        print("Dropping columns due to nans > 50%:", df0.loc[:, list((100 * (df0.isnull().sum() / len(df0.index)) > 50))].columns)
        df0 = df0.drop(df0.loc[:, list((100 * (df0.isnull().sum() / len(df0.index)) > 50))].columns, 1)
        df0 = df0.ffill().bfill()

        print("Any columns still contain nans:", df0.isnull().values.any())

        df_returns = pd.DataFrame()
        for name in df0.columns:
          df_returns[name] = np.log(df0[name]).diff()

        #df_returns['SPY_PRICE'] = df0['SPY']
        #df_returns[self.config.stocks_adj_close_names] = df0[self.config.stocks]
        # split into train and test
        df_returns.dropna(axis=0, how='any', inplace=True)


        train_data = df_returns.iloc[:-self.Ntest]
        test_data = df_returns.iloc[-self.Ntest:]


        return train_data, test_data

ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


ModuleNotFoundError: No module named 'gym'

In [ ]:
class EnvConfig():
    def __init__(self):
        self.stocks = ['AAPL', 'MSFT', 'AMZN', 'NFLX', 'XOM', 'JPM', 'T']
        self.initial_cash = 10_000
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
"""
Aktionen;
0: Halten
1: Kaufen
2: Verkaufen
"""
class MyEnv(gym.Env):

    def __init__(self, df_train, df_test, play=False):
        self.df_train = df_train
        self.df_test = df_test
        self.play = play
        if not self.play:
            self.df = self.df_train
        else:
            self.df = self.df_test
        self.config = EnvConfig()
        self.current_idx = 0
        self.stocks = ['AAPL', 'MSFT', 'AMZN', 'NFLX', 'XOM', 'JPM', 'T'] # target stocks
        #self.stocks_adj_close_names = [stock + '_Adj_Close' for stock in self.stocks] # AAPL_Adj_Close so heissen die Spalten bei mir ?

        self.weights = np.full((len(self.config.stocks)), 1 / len(self.config.stocks), dtype=float)

        # cash per stock_values
        self.initial_cash = self.weights * self.config.initial_cash

        self.portfolio_value = 0
        self.stock_values = np.zeros(len(self.stocks))

        self.states = self.df.loc[:, ~self.df.columns.isin(self.stocks)].to_numpy()  # aufpassen, tatsächlich existierende Spaltennamen
        self.rewards = self.df[self.stocks].to_numpy()
        #self.n = len(self.states)
        self.n = len(self.df)


    def reset(self):
        self.cash = self.initial_cash
        self.portfolio_value = 0
        self.stock_values = np.zeros(len(self.config.stocks))

        self.current_idx = 0

        next_state = self.states[self.current_idx]
        next_state = np.array(next_state).reshape(1, -1)
        next_state = torch.tensor(next_state).float().to(self.config.device)
        return next_state

    def step(self, action):
        reward = 0

        if self.current_idx >= self.n:
            raise Exception("Episode already done")


        # state transition
        done = (self.current_idx == self.n - 1)

        # apply action todo
        probs = [1,1,1]
        action = np.random.choice([0,1,2], probs)

        self.current_idx += 1

        if not done:

            # compute reward todo
            previous = np.sum(self.states[self.current_idx-1])
            next = np.sum(self.states[self.current_idx])
            reward = (next-previous)/previous


            next_state = self.states[self.current_idx]
            next_state = np.array(next_state).reshape(1, -1)
            #next_state = torch.tensor(next_state).reshape(1, -1).float().to(self.config.device)

        else:
            next_state = None
            reward = 0
            #self.portfolio_value = np.dot(self.stock_values, current_prices)

        return next_state, reward, done

In [ ]:
dataset = MyDataset()
train, test = dataset.get_train_test()

In [ ]:
train_env = MyEnv(train,test,False)
train_env.reset()

print(train_env.n)
next_state, reward, done = train_env.step(1)
print(next_state)
next_state, reward, done = train_env.step(1)
#print(next_state)